# Document Policy Compliance Checker with Human-in-the-Loop

## What You'll Learn

In this notebook, you'll build an automated compliance checker that reviews documents against organizational policies. More importantly, you'll learn about **human-in-the-loop (HIL)** workflows and understand why they're essential for real-world AI applications.

## Why Human-in-the-Loop Matters

Imagine you work in an organization where policy changes happen frequently. You might need to review hundreds of documents to ensure they all comply with new requirements. An AI agent could scan these documents automatically, but what if it makes a mistake? What if a document is ambiguous and could be interpreted multiple ways?

**Human-in-the-loop workflows solve this problem** by allowing an AI agent to:
1. Do the tedious work of scanning and flagging potential issues
2. **Pause and ask for human confirmation** before taking action
3. Continue processing based on human feedback

This combines the efficiency of automation with the judgment of human expertise.

## What is LangGraph?

**LangGraph** is a framework for building stateful, multi-step AI applications (often called "agents"). Think of it as a way to create workflows where:
- Your AI can perform multiple steps in sequence
- The AI can remember what happened in previous steps
- You can pause the workflow and resume it later
- You can insert human decision points exactly where you need them

It's particularly powerful for tasks that require multiple decisions or where you need to maintain context across several operations.

## Our Scenario

We'll build a policy compliance checker that verifies official documents contain specific metadata meeting these requirements:

- **Title**: Required, cannot be blank
- **Last Updated**: Required, must be in YYYY-MM-DD format
- **Reviewed By**: Required, must contain at least one reviewer name (first and last)
- **Status**: Required, must be one of: Draft, Final, or Archived
- **Version**: Required, must be in semantic version format (e.g., 1.0.0)

We'll build this in two stages:
1. **Pattern matching version**: Fast, simple, but very rigid
2. **LLM-enhanced version**: Smarter at handling ambiguity, but still needs human oversight

## Installation and Setup

First, let's install the packages we need using conda. We're using:
- **langgraph**: The framework for building our stateful workflow
- **langchain_anthropic**: To connect to Claude for the LLM-enhanced version

Both packages are available on conda-forge, which is the community-led collection of recipes for the conda package manager.

In [1]:
# Install required packages using conda
# The -y flag automatically confirms the installation and the -q flag makes conda run quietly
%conda install -c conda-forge langgraph langchain-anthropic -y -q

2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries needed throughout the notebook
import os
import getpass
import re
from typing import Annotated
from pathlib import Path

# LangGraph imports - each will be explained as it is used
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

### API Key Setup (Optional for Part 1)

**Note**: You don't need an API key to run Part 1 (pattern matching version). You'll only need an Anthropic API key for Part 2 if you want to try the LLM-enhanced version.

To get an API key:
1. Sign up at https://console.anthropic.com
2. Navigate to API Keys
3. Create a new key

In [3]:
# This function safely prompts for an API key if one isn't already set
def set_api_key(key_name: str):
    """Prompt for API key if not already set in environment."""
    if not os.environ.get(key_name):
        os.environ[key_name] = getpass.getpass(f"{key_name}: ")

# Uncomment the line below when you're ready to try Part 2
set_api_key("ANTHROPIC_API_KEY")

## Creating Sample Documents

Let's create a few sample documents that represent different compliance scenarios. We'll save these as markdown files so you can easily see their content and modify them if you want to experiment.

We'll create:
1. A **compliant** document that meets all requirements
2. A document with **missing fields**
3. A document with an **ambiguous entry** that could be interpreted different ways

In [4]:
# Create a directory to store our sample documents
docs_dir = Path("sample_documents")
docs_dir.mkdir(exist_ok=True)

# Document 1: Fully compliant
compliant_doc = """---
Title: Data Retention Policy
Last Updated: 2024-11-01
Reviewed By: Sarah Chen
Status: Final
Version: 2.1.0
---

# Data Retention Policy

This policy outlines the requirements for data retention across all departments.
"""

# Document 2: Missing required fields
missing_fields_doc = """---
Title: Employee Handbook
Last Updated: 2024-10-15
---

# Employee Handbook

Welcome to the company! This handbook contains important information.
"""

# Document 3: Ambiguous Reviewer Name (human judgment needed)
ambiguous_name_doc = """---
Title: Security Protocols
Last Updated: 2024-09-20
Reviewed By: Legal Team
Status: Final
Version: 3.2.1
---

# Security Protocols

Critical security measures for all staff.
"""

# Save all documents to files
documents = {
    "data_retention.md": compliant_doc,
    "employee_handbook.md": missing_fields_doc,
    "security_protocols.md": ambiguous_name_doc
}

for filename, content in documents.items():
    filepath = docs_dir / filename
    filepath.write_text(content)
    print(f"Created: {filepath}")

Created: sample_documents/data_retention.md
Created: sample_documents/employee_handbook.md
Created: sample_documents/security_protocols.md


## Part 1: Pattern Matching Version

Let's start with a simple version that uses pattern matching (regular expressions) to check compliance. This approach is:
- **Fast**: No API calls needed
- **Deterministic**: Same input always gives same output
- **Free**: No API costs

But it's also **rigid**: It can't understand context or handle ambiguity.

### Understanding LangGraph Concepts

Before we build our workflow, let's understand the key concepts:

#### 1. State
**State** is the data that flows through your workflow. Think of it as a shared notebook that every step can read from and write to. In our case, the state will track:
- Which documents we're checking
- What violations we've found
- What the human decided about flagged documents

#### 2. Nodes
**Nodes** are the individual steps in your workflow. Each node is a function that:
- Receives the current state
- Does some work
- Returns updates to the state

#### 3. Edges
**Edges** connect nodes together, defining the flow of execution. They determine which node runs next.

#### 4. Checkpointer
A **checkpointer** saves the state at each step. This is crucial for human-in-the-loop workflows because it lets us:
- Pause execution
- Wait for human input
- Resume exactly where we left off

Think of it like saving your progress in a video game.

#### 5. Interrupt
The **interrupt()** function pauses execution and waits for human input. It's the key to human-in-the-loop workflows.

### Defining Our State

First, let's define what information our workflow needs to track:

In [5]:
from typing import TypedDict

class ComplianceState(TypedDict):
    """State that tracks our compliance checking workflow.
    
    This dictionary will be passed between all nodes in our graph.
    Each node can read from it and update it.
    """
    # List of document filenames to check
    documents_to_check: list[str]
    
    # Index of the current document we're processing
    current_document_index: int
    
    # Results of our compliance checks
    # Each entry is a dict with: filename, compliant (bool), issues (list)
    results: list[dict]
    
    # When we find a potential violation, we store it here to show the human
    pending_review: dict | None
    
    # Whether we're finished checking all documents
    complete: bool

### Building the Compliance Checker Function

Now let's create a function that checks if a document meets our policy requirements using pattern matching:

In [6]:
def check_document_compliance_pattern_matching(filepath: Path) -> dict:
    """Check if a document meets compliance requirements using pattern matching.
    
    Args:
        filepath: Path to the markdown document to check
        
    Returns:
        dict with keys:
            - filename: Name of the file
            - compliant: True if all requirements met
            - issues: List of compliance issues found
            - metadata: The extracted metadata
    """
    content = filepath.read_text()
    issues = []
    
    # Extract the metadata section (between the --- markers)
    metadata_match = re.search(r'^---\s*\n(.*?)\n---', content, re.MULTILINE | re.DOTALL)
    
    if not metadata_match:
        return {
            "filename": filepath.name,
            "compliant": False,
            "issues": ["No metadata section found"],
            "metadata": {}
        }
    
    metadata_text = metadata_match.group(1)
    
    # Parse metadata into a dictionary
    metadata = {}
    for line in metadata_text.split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            metadata[key.strip()] = value.strip()
    
    # Check each requirement
    
    # 1. Title: Required and cannot be blank
    if 'Title' not in metadata:
        issues.append("Missing required field: Title")
    elif not metadata['Title'] or metadata['Title'].strip() == '':
        issues.append("Title field cannot be blank")
    
    # 2. Last Updated: Required and must be YYYY-MM-DD format
    if 'Last Updated' not in metadata:
        issues.append("Missing required field: Last Updated")
    else:
        # Check for YYYY-MM-DD format using regex
        date_pattern = r'^\d{4}-\d{2}-\d{2}$'
        if not re.match(date_pattern, metadata['Last Updated']):
            issues.append(f"Last Updated must be in YYYY-MM-DD format (found: '{metadata['Last Updated']}')")
    
    # 3. Reviewed By: Required and must have at least one name
    if 'Reviewed By' not in metadata:
        issues.append("Missing required field: Reviewed By")
    elif not metadata['Reviewed By'] or metadata['Reviewed By'].strip() == '':
        issues.append("Reviewed By cannot be blank")
    else:
        # Check that Reviewed By has at least two words (first and last name)
        if len(metadata['Reviewed By'].split()) < 2:
            issues.append("Reviewed By must include at least a first and last name")
    
    # 4. Status: Required and must be Draft, Final, or Archived
    valid_statuses = ['Draft', 'Final', 'Archived']
    if 'Status' not in metadata:
        issues.append("Missing required field: Status")
    elif metadata['Status'] not in valid_statuses:
        issues.append(f"Status must be one of {valid_statuses} (found: '{metadata['Status']}')")
    
    # 5. Version: Required and should be semantic version format
    if 'Version' not in metadata:
        issues.append("Missing required field: Version")
    else:
        # Check for semantic version format (e.g., 1.0.0)
        version_pattern = r'^\d+\.\d+\.\d+$'
        if not re.match(version_pattern, metadata['Version']):
            issues.append(f"Version should be in semantic version format like 1.0.0 (found: '{metadata['Version']}')")
    
    return {
        "filename": filepath.name,
        "compliant": len(issues) == 0,
        "issues": issues,
        "metadata": metadata
    }

Let's test this function on one of our documents to see how it works:

In [7]:
# Test on the missing fields document
test_result = check_document_compliance_pattern_matching(docs_dir / "employee_handbook.md")
print(f"Document: {test_result['filename']}")
print(f"Compliant: {test_result['compliant']}")
print(f"Issues found: {test_result['issues']}")
print(f"Metadata: {test_result['metadata']}")

Document: employee_handbook.md
Compliant: False
Issues found: ['Missing required field: Reviewed By', 'Missing required field: Status', 'Missing required field: Version']
Metadata: {'Title': 'Employee Handbook', 'Last Updated': '2024-10-15'}


### Creating Our Workflow Nodes

Now let's create the nodes (steps) that make up our workflow:

In [8]:
def check_next_document(state: ComplianceState) -> ComplianceState:
    """Node that checks the next document in our queue.
    
    This node:
    1. Gets the next document to check
    2. Runs compliance checks on it
    3. If issues found, stores them for human review
    4. If no issues, marks it as compliant
    """
    current_idx = state["current_document_index"]
    documents = state["documents_to_check"]
    
    # Check if we've processed all documents
    if current_idx >= len(documents):
        state["complete"] = True
        return state
    
    # Get the current document and check it
    current_file = documents[current_idx]
    filepath = Path("sample_documents") / current_file
    
    print(f"\n Checking: {current_file}")
    
    result = check_document_compliance_pattern_matching(filepath)
    
    # If the document is not compliant, we need human review
    if not result["compliant"]:
        print(f"Issues found! Flagging for human review...")
        state["pending_review"] = result
    else:
        print(f"Document is compliant!")
        state["results"].append(result)
        # Move to next document
        state["current_document_index"] += 1
    
    return state


def human_review(state: ComplianceState) -> ComplianceState:
    """Node that pauses for human review of flagged documents.
    
    This is where the magic happens! We use interrupt() to pause execution
    and wait for a human decision.
    """
    pending = state["pending_review"]
    
    if pending is None:
        # Nothing to review, continue
        return state
    
    # Display the issues to the human
    print(f"\n Document '{pending['filename']}' flagged for review:")
    for issue in pending['issues']:
        print(f"   - {issue}")
    
    # Only show metadata if it exists (pattern matching version has it, LLM version doesn't)
    if 'metadata' in pending:
        print("\nMetadata found:")
        for key, value in pending['metadata'].items():
            print(f"   {key}: {value}")
    
    # If we have LLM analysis, show that instead
    if 'llm_analysis' in pending:
        print("\nLLM Analysis:")
        print(pending['llm_analysis'])
    
    # This is the key line! interrupt() pauses execution here.
    # The workflow won't continue until we resume it with a human decision.
    human_decision = interrupt(
        "Is this a compliance violation? Reply 'yes' to confirm or 'no' if this is acceptable."
    )
    
    print(f"\n Human decision: {human_decision}")
    
    # Record the decision
    if human_decision.lower() == 'yes':
        print("   → Marking as non-compliant")
        pending["human_confirmed"] = True
    else:
        print("   → Marking as compliant (false positive)")
        pending["compliant"] = True
        pending["issues"] = []  # Clear issues since human approved it
        pending["human_confirmed"] = False
    
    # Add to results and move to next document
    state["results"].append(pending)
    state["pending_review"] = None
    state["current_document_index"] += 1
    
    return state


def should_continue(state: ComplianceState) -> str:
    """Conditional edge that determines where to go next.
    
    This function decides the next step based on the current state:
    - If we're done checking all documents, end the workflow
    - If there's a document pending human review, go to human_review
    - Otherwise, check the next document
    """
    if state["complete"]:
        return "end"
    elif state["pending_review"] is not None:
        return "review"
    else:
        return "check"

### Building the Graph

Now we'll assemble all the pieces into a LangGraph workflow:

In [9]:
# Create the graph builder
# StateGraph takes our state type as a parameter
workflow = StateGraph(ComplianceState)

# Add our nodes to the graph
# The string names are how we'll reference these nodes when defining edges
workflow.add_node("check_document", check_next_document)
workflow.add_node("human_review", human_review)

# Define the starting point
# START is a special constant that marks where execution begins
workflow.add_edge(START, "check_document")

# Add conditional edges based on the should_continue function
# This function determines which node to go to next
workflow.add_conditional_edges(
    "check_document",
    should_continue,
    {
        "check": "check_document",  # Check another document
        "review": "human_review",    # Need human review
        "end": END                    # All done
    }
)

# After human review, go back to checking documents
workflow.add_edge("human_review", "check_document")

# Create a checkpointer to save our progress
# MemorySaver stores checkpoints in memory (good for demos)
# For production, you'd use a persistent storage backend
checkpointer = MemorySaver()

# Compile the graph into a runnable application
# The checkpointer parameter enables the pause/resume functionality
compliance_checker = workflow.compile(checkpointer=checkpointer)

print("Compliance checker workflow built successfully!")

Compliance checker workflow built successfully!


### Running the Compliance Checker

Now let's run our workflow! Watch how it:
1. Checks each document
2. Pauses when it finds issues
3. Waits for your decision
4. Continues based on your input

In [10]:
# Set up the initial state
initial_state = {
    "documents_to_check": [
        "data_retention.md",
        "employee_handbook.md",
        "security_protocols.md"
    ],
    "current_document_index": 0,
    "results": [],
    "pending_review": None,
    "complete": False
}

# Configuration for this run
# The thread_id identifies this particular workflow instance
# Think of it like a session ID - we can pause and resume using this ID
config = {"configurable": {"thread_id": "compliance-check-1"}}

print("Starting compliance check...\n")
print("="*60)

# Start the workflow
# This will run until it hits an interrupt() or completes
for event in compliance_checker.stream(initial_state, config):
    pass  # The nodes handle their own printing

Starting compliance check...


 Checking: data_retention.md
Document is compliant!

 Checking: employee_handbook.md
Issues found! Flagging for human review...

 Document 'employee_handbook.md' flagged for review:
   - Missing required field: Reviewed By
   - Missing required field: Status
   - Missing required field: Version

Metadata found:
   Title: Employee Handbook
   Last Updated: 2024-10-15


### Understanding What Just Happened

The workflow should have paused after finding the first non-compliant document. Let's check the current state:

In [11]:
# Get the current state of our workflow
current_state = compliance_checker.get_state(config)

print("Current workflow state:")
print(f"  - Documents checked so far: {current_state.values['current_document_index']}")
print(f"  - Waiting for human review: {current_state.values['pending_review'] is not None}")
print(f"  - Next node to execute: {current_state.next}")

Current workflow state:
  - Documents checked so far: 1
  - Waiting for human review: True
  - Next node to execute: ('human_review',)


### Providing Human Feedback

Now we can provide our decision ("yes" to agree that there is a policy violation) and let the workflow continue. We use `Command(resume=...)` to provide our answer to the `interrupt()` call:

In [12]:
# Provide human feedback: confirm this is a violation
print("\n" + "="*60)
print("Providing human feedback: 'yes' (confirming violation)\n")

# Resume the workflow with our decision
# Command(resume="yes") sends "yes" to the interrupt() call
for event in compliance_checker.stream(Command(resume="yes"), config):
    pass


Providing human feedback: 'yes' (confirming violation)


 Document 'employee_handbook.md' flagged for review:
   - Missing required field: Reviewed By
   - Missing required field: Status
   - Missing required field: Version

Metadata found:
   Title: Employee Handbook
   Last Updated: 2024-10-15

 Human decision: yes
   → Marking as non-compliant

 Checking: security_protocols.md
Document is compliant!


The workflow would pause again if it found another document with issues, but in this case our very rigid regex method was not able to detect that there was ambiguity with the metadata for our final document.

### Viewing the Final Results

Now let's look at our final compliance report:

In [13]:
# Get the final state
final_state = compliance_checker.get_state(config)
results = final_state.values["results"]

print("\n" + "="*60)
print("COMPLIANCE CHECK COMPLETE")
print("="*60)

compliant_count = sum(1 for r in results if r["compliant"])
total_count = len(results)

print(f"\nSummary: {compliant_count}/{total_count} documents compliant\n")

for result in results:
    status = "COMPLIANT" if result["compliant"] else "NON-COMPLIANT"
    print(f"{status}: {result['filename']}")
    
    if not result["compliant"]:
        print("  Issues:")
        for issue in result["issues"]:
            print(f"    - {issue}")
    
    # Show if human overrode the decision
    if "human_confirmed" in result:
        if result["human_confirmed"]:
            print("  Human confirmed violation")
        else:
            print("  Human marked as false positive")
    
    print()


COMPLIANCE CHECK COMPLETE

Summary: 2/3 documents compliant

COMPLIANT: data_retention.md

NON-COMPLIANT: employee_handbook.md
  Issues:
    - Missing required field: Reviewed By
    - Missing required field: Status
    - Missing required field: Version
  Human confirmed violation

COMPLIANT: security_protocols.md



### Limitations of Pattern Matching

Notice how our pattern matching approach was unable to recognize "Legal Team" as non-compliant even though a human might recognize this as an invalid name? This is where an LLM could help!

Pattern matching is:
- Fast and free
- Predictable and deterministic
- Can't understand context
- Can't handle ambiguity

Let's see how we can improve this with an LLM...

## Part 2: LLM-Enhanced Version

Now let's upgrade our compliance checker to use Claude (an LLM) for analysis. This will help us:
- Handle ambiguous cases better
- Understand context and intent
- Provide more nuanced analysis

**But here's the key insight**: Even with an LLM, we still use human-in-the-loop! Why?
- LLMs can still make mistakes
- Compliance decisions often have legal/business implications
- Humans provide accountability and final judgment

### Setting Up the LLM

In [14]:
# Make sure you've set your API key (run this if you haven't already)
set_api_key("ANTHROPIC_API_KEY")

from langchain_anthropic import ChatAnthropic

# Initialize Claude
# We're using claude-sonnet-4-5-20250929 which is fast and cost-effective
llm = ChatAnthropic(
    model="claude-sonnet-4-5-20250929",
    temperature=0  # Low temperature for consistent, deterministic responses
)

print("LLM initialized")

LLM initialized


### Creating an LLM-Based Checker

Let's create a new compliance checker function that uses Claude to analyze documents:

In [46]:
def check_document_compliance_llm(filepath: Path) -> dict:
    """Check document compliance using an LLM for more nuanced analysis.
    
    The LLM can understand context and handle ambiguity better than
    simple pattern matching.
    """
    content = filepath.read_text()
    
    # Create a prompt that explains our requirements clearly
    prompt = f"""You are a document compliance analyst. Review the following document and check if it meets these metadata requirements:

REQUIRED METADATA:
1. Title: Must be present and cannot be blank
2. Last Updated: Must be present in YYYY-MM-DD format (e.g., 2024-11-01)
3. Reviewed By: Must be present with at least one reviewer name (first and last)
4. Status: Must be present and be one of: Draft, Final, or Archived
5. Version: Must be present in semantic version format (e.g., 1.0.0)

DOCUMENT TO REVIEW:
{content}

Analyze this document and respond in this exact format:

COMPLIANT: [yes or no]
ISSUES:
- [list each compliance issue on a separate line, or write "None" if compliant]

Use good judgment for ambiguous cases. For example, a Title that is just letters that do not form words would not be compliant.
"""
    
    # Call the LLM
    response = llm.invoke(prompt)
    response_text = response.content
    
    # Parse the LLM's response
    compliant = "COMPLIANT: yes" in response_text
    
    # Extract issues from the response
    issues = []
    if "ISSUES:" in response_text:
        issues_section = response_text.split("ISSUES:")[1].strip()
        issues = [
            line.strip("- ").strip() 
            for line in issues_section.split("\n") 
            if line.strip() and line.strip() != "None"
        ]
    
    return {
        "filename": filepath.name,
        "compliant": compliant,
        "issues": issues,
        "llm_analysis": response_text
    }

Let's test this on our ambiguous name document to see how the LLM handles it:

In [47]:
# Test on the ambiguous document
llm_result = check_document_compliance_llm(docs_dir / "security_protocols.md")
#print(f"Document: {llm_result['filename']}")
#print(f"Compliant: {llm_result['compliant']}")
#print(f"Issues: {llm_result['issues']}")
print(f"\nFull LLM Analysis:\n{llm_result['llm_analysis']}")


Full LLM Analysis:
COMPLIANT: no
ISSUES:
- Reviewed By: "Legal Team" does not include a specific reviewer name with first and last name (e.g., "John Smith"). A team name is not sufficient to meet the requirement of "at least one reviewer name (first and last)"


The LLM was able to successfully determine that "Legal Team" is not a valid name!

### Building the LLM-Enhanced Workflow

Now let's create a new version of our workflow that uses the LLM checker:

In [48]:
def check_next_document_llm(state: ComplianceState) -> ComplianceState:
    """LLM-enhanced version of document checking node."""
    current_idx = state["current_document_index"]
    documents = state["documents_to_check"]
    
    if current_idx >= len(documents):
        state["complete"] = True
        return state
    
    current_file = documents[current_idx]
    filepath = Path("sample_documents") / current_file
    
    print(f"\n Checking with LLM: {current_file}")
    
    # Use LLM-based checking instead of pattern matching
    result = check_document_compliance_llm(filepath)
    
    if not result["compliant"]:
        print(f"Issues found! Flagging for human review...")
        state["pending_review"] = result
    else:
        print(f"Document is compliant!")
        state["results"].append(result)
        state["current_document_index"] += 1
    
    return state

# Build the new workflow (same structure, different checker)
workflow_llm = StateGraph(ComplianceState)
workflow_llm.add_node("check_document", check_next_document_llm)
workflow_llm.add_node("human_review", human_review)  # Reuse same review node

workflow_llm.add_edge(START, "check_document")
workflow_llm.add_conditional_edges(
    "check_document",
    should_continue,  # Reuse same conditional logic
    {
        "check": "check_document",
        "review": "human_review",
        "end": END
    }
)
workflow_llm.add_edge("human_review", "check_document")

# Create a new checkpointer for this workflow
checkpointer_llm = MemorySaver()
compliance_checker_llm = workflow_llm.compile(checkpointer=checkpointer_llm)

print("LLM-enhanced compliance checker workflow built successfully!")

LLM-enhanced compliance checker workflow built successfully!


### Running the LLM-Enhanced Checker

Let's run the same documents through our LLM-enhanced workflow:

In [49]:
# Reset state for the new run
initial_state_llm = {
    "documents_to_check": [
        "data_retention.md",
        "employee_handbook.md",
        "security_protocols.md"
    ],
    "current_document_index": 0,
    "results": [],
    "pending_review": None,
    "complete": False
}

# Fresh thread ID
config_llm = {"configurable": {"thread_id": "compliance-check-llm-6"}}

print("Starting LLM-enhanced compliance check...\n")
print("="*60)

for event in compliance_checker_llm.stream(initial_state_llm, config_llm):
    pass

Starting LLM-enhanced compliance check...


 Checking with LLM: data_retention.md
Document is compliant!

 Checking with LLM: employee_handbook.md
Issues found! Flagging for human review...

 Document 'employee_handbook.md' flagged for review:
   - Reviewed By: Missing - no reviewer name present
   - Status: Missing - must be Draft, Final, or Archived
   - Version: Missing - must be in semantic version format (e.g., 1.0.0)

LLM Analysis:
COMPLIANT: no
ISSUES:
- Reviewed By: Missing - no reviewer name present
- Status: Missing - must be Draft, Final, or Archived
- Version: Missing - must be in semantic version format (e.g., 1.0.0)


Provide feedback as needed (same process as before):

In [50]:
# Provide feedback for any flagged documents
# You'll need to run this cell for each document that gets flagged
print("\n" + "="*60)
print("Providing human feedback: 'yes'\n")

for event in compliance_checker_llm.stream(Command(resume="yes"), config_llm):
    pass


Providing human feedback: 'yes'


 Document 'employee_handbook.md' flagged for review:
   - Reviewed By: Missing - no reviewer name present
   - Status: Missing - must be Draft, Final, or Archived
   - Version: Missing - must be in semantic version format (e.g., 1.0.0)

LLM Analysis:
COMPLIANT: no
ISSUES:
- Reviewed By: Missing - no reviewer name present
- Status: Missing - must be Draft, Final, or Archived
- Version: Missing - must be in semantic version format (e.g., 1.0.0)

 Human decision: yes
   → Marking as non-compliant

 Checking with LLM: security_protocols.md
Issues found! Flagging for human review...

 Document 'security_protocols.md' flagged for review:
   - Reviewed By: "Legal Team" does not include a specific reviewer name with first and last name (e.g., "John Smith"). A team name is not sufficient to meet the requirement of "at least one reviewer name (first and last)"

LLM Analysis:
COMPLIANT: no
ISSUES:
- Reviewed By: "Legal Team" does not include a specific reviewer 

In [51]:
# Continue providing feedback for remaining documents
print("\n" + "="*60)
print("Providing human feedback: 'yes'\n")

for event in compliance_checker_llm.stream(Command(resume="yes"), config_llm):
    pass


Providing human feedback: 'yes'


 Document 'security_protocols.md' flagged for review:
   - Reviewed By: "Legal Team" does not include a specific reviewer name with first and last name (e.g., "John Smith"). A team name is not sufficient to meet the requirement of "at least one reviewer name (first and last)"

LLM Analysis:
COMPLIANT: no
ISSUES:
- Reviewed By: "Legal Team" does not include a specific reviewer name with first and last name (e.g., "John Smith"). A team name is not sufficient to meet the requirement of "at least one reviewer name (first and last)"

 Human decision: yes
   → Marking as non-compliant


### Comparing Results

Let's see how the LLM version performed compared to pattern matching:

In [52]:
final_state_llm = compliance_checker_llm.get_state(config_llm)
results_llm = final_state_llm.values["results"]

print("\n" + "="*60)
print("LLM-ENHANCED COMPLIANCE CHECK COMPLETE")
print("="*60)

compliant_count_llm = sum(1 for r in results_llm if r["compliant"])
total_count_llm = len(results_llm)

print(f"\nSummary: {compliant_count_llm}/{total_count_llm} documents compliant\n")

for result in results_llm:
    status = "COMPLIANT" if result["compliant"] else "NON-COMPLIANT"
    print(f"{status}: {result['filename']}")
    
    if not result["compliant"]:
        print("  Issues:")
        for issue in result["issues"]:
            print(f"    - {issue}")
    
    if "human_confirmed" in result:
        if result["human_confirmed"]:
            print("  Human confirmed violation")
        else:
            print("  Human marked as false positive")
    
    print()


LLM-ENHANCED COMPLIANCE CHECK COMPLETE

Summary: 1/3 documents compliant

COMPLIANT: data_retention.md

NON-COMPLIANT: employee_handbook.md
  Issues:
    - Reviewed By: Missing - no reviewer name present
    - Status: Missing - must be Draft, Final, or Archived
    - Version: Missing - must be in semantic version format (e.g., 1.0.0)
  Human confirmed violation

NON-COMPLIANT: security_protocols.md
  Issues:
    - Reviewed By: "Legal Team" does not include a specific reviewer name with first and last name (e.g., "John Smith"). A team name is not sufficient to meet the requirement of "at least one reviewer name (first and last)"
  Human confirmed violation



## Key Takeaways

### Why Human-in-the-Loop Matters

1. **Accountability**: For compliance and policy enforcement, humans must remain in the decision-making loop
2. **Judgment**: Ambiguous cases often require context that only humans have
3. **Trust**: Users trust systems more when they can review and override decisions
4. **Learning**: Human feedback helps improve the system over time

### Pattern Matching vs. LLM

**Pattern Matching**:
- Pros: Fast, free, deterministic, good for clear-cut rules
- Cons: Can't handle ambiguity

**LLM-Enhanced**:
- Pros: Capable of more nuanced analysis, can understand context
- Cons:Costs money (API calls), potentially slower

### Real-World Applications

This pattern works for many scenarios:
- **Legal/Compliance**: Reviewing contracts, policies, or regulations
- **Content Moderation**: Flagging problematic content for human review
- **Quality Assurance**: Checking code, documentation, or data quality
- **Medical/Healthcare**: Flagging cases that need expert review
- **Financial**: Reviewing transactions or documents for approval

### What You've Learned

1. **LangGraph fundamentals**: State, nodes, edges, checkpointers
2. **Human-in-the-loop workflows**: Using `interrupt()` and `Command(resume=...)`
3. **Iterative development**: Starting simple, then add complexity
4. **When to use LLMs**: Trade-offs between pattern matching and LLM analysis
5. **Why human oversight matters**: Even smart AI can benefit from human judgment

## Next Steps

Want to extend this project? Try:

1. **Add more document types**: Support .docx, .pdf, or other formats
2. **Implement batch processing**: Check entire directories of documents
3. **Add a reporting feature**: Generate PDF or HTML compliance reports
4. **Persistent storage**: Use a database to store checkpoints and results
5. **Multi-user support**: Allow different reviewers to handle different document types
6. **Policy templates**: Allow users to define custom compliance rules
7. **Integration**: Connect to document management systems or Slack for notifications

## Learn More

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Anthropic API Documentation](https://docs.anthropic.com)
- [Search for LangGraph on anaconda.org](https://anaconda.org/search?q=langgraph) to see package details, download stats, and more!